In [406]:
import torch
import os
import argparse
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from functions import load_simple_scenarios_with_flexible_context
from pald_implementation import (
    make_pald_base_layer,
    make_pald_flex_purchase_layer,
    make_pald_flex_delivery_layer,
    compute_segment_caps
)
from paad_implementation import get_alpha
from robust_projection import project_y_robust, project_y_flex_robust
from contextual_model import ThresholdPredictor
import paad_implementation as pi
import math
from paad_implementation import objective_function as np_objective_function
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import numpy as np
import pickle
import opt_sol
from tqdm import tqdm
from typing import Dict, List, Tuple


In [407]:
K = 10           # number of segments in piecewise linear approximation for psi
gamma = 1.0     # switching cost parameter for x
delta = 0.5     # switching cost parameter for z (used in analytical threshold)
S = 1.0          # maximum inventory capacity
c_delivery = 0.2
eps_delivery = 0.05
epochs = 10
T = 10

In [408]:
# define a toy instance of osdm where prices decrease from 10 to 1 and then jump up to 10, 
# single unit of base demand at T=10, no flex demand
p_min = 1.0
p_max = 10.0
base_demand_all = [1.0 if t == T-1 else 0.0 for t in range(T)]
flex_demand_all = [0.0 for t in range(T)]

prices = np.linspace(p_max, p_min, T-1).tolist()
prices.append(p_max)

In [409]:
solver_options = {
    # SCS parameters tend to be robust for differentiable layers
    "eps": 1e-5,
    "max_iters": 2000,
    "verbose": False,
}


def _safe_layer_call(layer, args, size=1.0):
    """
    Call a CvxpyLayer and catch SCS/diffcp failures. Returns x_total tensor.
    """
    x_total, x_parts = layer(*args, solver_args=solver_options)
    return x_total

In [410]:
def make_pald_base_layer(K, gamma, ridge=False):
    x_parts = cp.Variable(K, nonneg=True)
    x_total = cp.Variable(nonneg=True)

    x_prev = cp.Parameter(nonneg=True)
    w_prev = cp.Parameter(nonneg=True)
    p_t = cp.Parameter()
    y_vec = cp.Parameter((K,))
    caps = cp.Parameter((K,), nonneg=True)

    constraints = [
        x_parts >= 0,
        x_parts <= caps,
        x_total == cp.sum(x_parts),
        x_total <= 1 - w_prev,
    ]
    ridge = 0
    if ridge:
        # Increase ridge to encourage interior solutions and smoother differentiability
        ridge = 1e-3 * cp.sum_squares(x_parts) + 1e-3 * cp.sum_squares(x_total)
    hit_cost = p_t * x_total
    switch_cost = gamma * cp.abs(x_total - x_prev) + gamma * cp.abs(x_total)
    phi_cost = y_vec @ x_parts
    # write down the closed-form of the messy piecewise quadratic integral - increasing and concave 
    # decomposing into x_parts -- assumptions -- double check (discontinuities) -- define as min or max of quadratic functions -- don't need to do this decomposition into x_part
    
    # each x_part corresponds to a quadratic function
    obj = cp.Minimize(hit_cost + switch_cost - phi_cost + ridge)
    prob = cp.Problem(obj, constraints)
    return CvxpyLayer(prob,
                      parameters=[x_prev, w_prev, p_t, y_vec, caps],
                      variables=[x_total, x_parts])

In [411]:
# helper to compute a (coarse approximation) of the integral over the (piecewise-affine) threshold function phi
def compute_segment_caps(w_prev: float, K: int):
    """Remaining capacity per segment given cumulative fraction w_prev."""
    # Clamp w into [0, 1]
    w = max(0.0, min(1.0, float(w_prev)))
    if 1.0 - w <= 1e-9:
        return [0.0] * K
    caps = []
    for i in range(K):
        left = i / K
        right = (i + 1) / K
        cap = max(0.0, right - max(left, w))
        caps.append(cap)
    return caps

In [412]:
# differentiable torch objective function
# NOTE: Keep everything as torch ops; avoid Python floats that can break the graph.
def torch_objective(p_seq, x_seq, z_seq, gamma, delta, c, eps):
    """Torch version of objective_function for differentiable PALD cost.
    Inputs are torch 1D tensors of length T (float32).
    Mirrors paad_implementation.objective_function.
    """
    Tn = p_seq.shape[0]
    # state of charge s[0..T]
    s = []
    s_prev = torch.zeros(1, dtype=p_seq.dtype, device=p_seq.device)
    s.append(s_prev)
    for t in range(1, Tn + 1):
        s_t = torch.clamp(s_prev + x_seq[t - 1] - z_seq[t - 1], min=0.0)
        s.append(s_t)
        s_prev = s_t
    s_torch = torch.cat(s, dim=0)

    # Costs
    cost_purchasing = (p_seq * x_seq).sum()
    switching_cost_x = gamma * (x_seq[1:] - x_seq[:-1]).abs().sum() if Tn > 1 else torch.tensor(0.0)
    switching_cost_z = delta * (z_seq[1:] - z_seq[:-1]).abs().sum() if Tn > 1 else torch.tensor(0.0)
    s_prev_seq = s_torch[:-1]
    discharge_cost = (p_seq * (c * z_seq + eps * z_seq - c * s_prev_seq * z_seq)).sum()
    return cost_purchasing + switching_cost_x + switching_cost_z + discharge_cost

In [413]:
# define the base demand cvxpylayer
pald_base_layer = make_pald_base_layer(K, gamma, ridge=False)

In [414]:
# y_vec is the values of the piecewise affine threshold function -- let's see what happens if we set it HIGH
# IMPORTANT: To get valid gradients w.r.t. y_vec, we must keep decisions as torch Tensors and avoid .item()/.round()
y_vec_large = torch.tensor([p_max for i in range(K)], dtype=torch.float32, requires_grad=True)


def simulate_one_driver(y_vec, prices, base_demand_all, flex_demand_all, ridge=False):
    """
    Simulate one base driver with given y_vec and return a differentiable loss.
    Decisions remain as torch.Tensors so autograd can backprop to y_vec through the CvxpyLayer.
    """
    # torch state (kept differentiable across time)
    x_prev = torch.tensor(0.0, dtype=torch.float32)  # previous decision (scalar)
    w_prev = torch.tensor(0.0, dtype=torch.float32)  # cumulative fraction delivered

    torch_decisions = []

    for t in range(T):
        p_t = prices[t]
        # Prepare parameters for the layer
        x_prev_t = x_prev.view(1)
        w_prev_t = w_prev.view(1)
        p_t_t = torch.tensor([p_t], dtype=torch.float32)

        # Segment caps are not differentiated; treat them as constants derived from current w_prev
        caps_list = compute_segment_caps(float(w_prev.detach().cpu().item()), K)
        caps_t = torch.tensor(caps_list, dtype=torch.float32)

        # Solve the per-step convex problem via CVXPYLayer (returns x_total, x_parts)
        x_total_t = _safe_layer_call(
            pald_base_layer, (x_prev_t, w_prev_t, p_t_t, y_vec, caps_t)
        )
        # Squeeze to scalar for arithmetic
        x_total_scalar = x_total_t.squeeze()

        # On the final step, force completion if any remaining fraction < 1.0
        if t == T - 1:
            # Add just enough slack to finish any remainder (keeps graph intact)
            remainder = torch.clamp(1.0 - (w_prev + x_total_scalar), min=0.0)
            x_total_scalar = x_total_scalar + remainder

        torch_decisions.append(x_total_scalar)

        # Update state for the next step (kept differentiable)
        w_prev = w_prev + x_total_scalar
        x_prev = x_total_scalar

    # Stack decisions and compute differentiable objective
    x_seq = torch.stack(torch_decisions)  # shape [T]
    z_seq = torch.tensor(base_demand_all, dtype=torch.float32)  # no flex here; constant

    loss = torch_objective(
        torch.tensor(prices, dtype=torch.float32),
        x_seq,
        z_seq,
        gamma,
        delta,
        c_delivery,
        eps_delivery,
    )

    # For visibility only (no graph break for the loss)
    with torch.no_grad():
        pretty = [round(float(v), 4) for v in x_seq]
        print("Decisions:", pretty)
        print("Cost of this solution:", float(loss))

    return loss


# Run and backprop to inspect gradients w.r.t y_vec
loss = simulate_one_driver(y_vec_large, prices, base_demand_all, flex_demand_all)
loss.backward()
print("Gradients of cost w.r.t. y_vec (large init):", y_vec_large.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_large - lr * y_vec_large.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [0.0, 0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Cost of this solution: 10.75
Gradients of cost w.r.t. y_vec (large init): tensor([6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09,
        6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09])
New y_vec after one gradient step: tensor([9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936,
        9.9936], grad_fn=<SubBackward0>)


In [415]:
# what if we make y_vec small?
y_vec_small = torch.tensor([p_min for i in range(K)], dtype=torch.float32, requires_grad=True)

loss_small = simulate_one_driver(y_vec_small, prices, base_demand_all, flex_demand_all)
loss_small.backward()
print("Gradients of cost w.r.t. y_vec (small init):", y_vec_small.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_small - lr * y_vec_small.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 1.0]
Cost of this solution: 14.0
Gradients of cost w.r.t. y_vec (small init): tensor([3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10,
        3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10])
New y_vec after one gradient step: tensor([0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996,
        0.9996], grad_fn=<SubBackward0>)


In [416]:
# the above is a big problem because the threshold is now outside the range [p_min, p_max] !

In [417]:
# what is the optimal solution?
flex_demand_all = [0.0 for t in range(T)]
Deltas = [0.0 for t in range(T)]
status, results = opt_sol.optimal_solution(T, prices, gamma, delta, c_delivery, eps_delivery, S, base_demand_all, flex_demand_all, Deltas)
if status == "Optimal" and results is not None:
    decisions = results['x']
    opt_z = results['z']
    opt_s = results['s'][1:]
    # Use numpy objective for consistency
    opt_cost = np_objective_function(T, prices, gamma, delta, c_delivery, eps_delivery, decisions, opt_z)
decisions = [round(d, 4) for d in decisions]
print("Optimal decisions: ", decisions)
print("Optimal cost: ", np_objective_function(T, prices, gamma, delta, c_delivery, eps_delivery, decisions, base_demand_all))

Optimal decisions:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0]
Optimal cost:  3.5625


In [418]:
# what if we add a penalty to encourage the drivers to "purchase early" (instead of relying on compulsory trade)

def simulate_one_driver_penalty(y_vec, prices, base_demand_all, flex_demand_all):
    """
    Simulate one base driver with given y_vec and return a differentiable loss.
    Decisions remain as torch.Tensors so autograd can backprop to y_vec through the CvxpyLayer.
    """
    # torch state (kept differentiable across time)
    x_prev = torch.tensor(0.0, dtype=torch.float32)  # previous decision (scalar)
    w_prev = torch.tensor(0.0, dtype=torch.float32)  # cumulative fraction delivered
    x_topup = torch.tensor(0.0, dtype=torch.float32)

    torch_decisions = []

    for t in range(T):
        p_t = prices[t]
        # Prepare parameters for the layer
        x_prev_t = x_prev.view(1)
        w_prev_t = w_prev.view(1)
        p_t_t = torch.tensor([p_t], dtype=torch.float32)

        # Segment caps are not differentiated; treat them as constants derived from current w_prev
        caps_list = compute_segment_caps(float(w_prev.detach().cpu().item()), K)
        caps_t = torch.tensor(caps_list, dtype=torch.float32)

        # Solve the per-step convex problem via CVXPYLayer (returns x_total, x_parts)
        x_total_t = _safe_layer_call(
            pald_base_layer, (x_prev_t, w_prev_t, p_t_t, y_vec, caps_t)
        )
        # Squeeze to scalar for arithmetic
        x_total_scalar = x_total_t.squeeze()

        # On the final step, force completion if any remaining fraction < 1.0
        if t == T - 1:
            # Add just enough slack to finish any remainder (keeps graph intact)
            remainder = torch.clamp(1.0 - (w_prev + x_total_scalar), min=0.0)
            x_total_scalar = x_total_scalar + remainder
            x_topup = remainder

        torch_decisions.append(x_total_scalar)

        # Update state for the next step (kept differentiable)
        w_prev = w_prev + x_total_scalar
        x_prev = x_total_scalar

    # Stack decisions and compute differentiable objective
    x_seq = torch.stack(torch_decisions)  # shape [T]
    z_seq = torch.tensor(base_demand_all, dtype=torch.float32)

    loss = torch_objective(
        torch.tensor(prices, dtype=torch.float32),
        x_seq,
        z_seq,
        gamma,
        delta,
        c_delivery,
        eps_delivery,
    )
    # add a penalty to the loss proportional to x_topup
    lamda = torch.tensor(200.0, dtype=torch.float32)
    loss += lamda * remainder

    # For visibility only (no graph break for the loss)
    with torch.no_grad():
        pretty = [round(float(v), 4) for v in x_seq]
        print("Decisions:", pretty)
        print("Augmented cost of this solution:", float(loss))

    return loss

In [419]:
y_vec_large = torch.tensor([p_max for i in range(K)], dtype=torch.float32, requires_grad=True)
# Run and backprop to inspect gradients w.r.t y_vec
loss = simulate_one_driver_penalty(y_vec_large, prices, base_demand_all, flex_demand_all)
loss.backward()
print("Gradients of augmented cost w.r.t. y_vec (large init):", y_vec_large.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_large - lr * y_vec_large.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [0.0, 0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Augmented cost of this solution: 10.75
Gradients of augmented cost w.r.t. y_vec (large init): tensor([6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09,
        6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09])
New y_vec after one gradient step: tensor([9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936,
        9.9936], grad_fn=<SubBackward0>)


In [420]:
# what if we make y_vec small?
y_vec_small = torch.tensor([p_min for i in range(K)], dtype=torch.float32, requires_grad=True)

loss_small = simulate_one_driver_penalty(y_vec_small, prices, base_demand_all, flex_demand_all)
loss_small.backward()
print("Gradients of cost w.r.t. y_vec (small init):", y_vec_small.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_small - lr * y_vec_small.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 1.0]
Augmented cost of this solution: 214.0
Gradients of cost w.r.t. y_vec (small init): tensor([6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08,
        6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08])
New y_vec after one gradient step: tensor([0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352,
        0.9352], grad_fn=<SubBackward0>)


In [421]:
# even if I add a large penalty to the compulsory trade (forced top-ups), the gradient is still pointing down?

# when I train using train_pald_contextual.py, this is one of the key issues I run into -- the NN learns that it should predict 

In [422]:
# retry with ridge enabled, but no penalty
# define the base demand cvxpylayer
pald_base_layer = make_pald_base_layer(K, gamma, ridge=True)

In [423]:
y_vec_large = torch.tensor([p_max for i in range(K)], dtype=torch.float32, requires_grad=True)
# Run and backprop to inspect gradients w.r.t y_vec
loss = simulate_one_driver(y_vec_large, prices, base_demand_all, flex_demand_all)
loss.backward()
print("Gradients of cost w.r.t. y_vec (large init):", y_vec_large.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_large - lr * y_vec_large.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [0.0, 0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Cost of this solution: 10.75
Gradients of cost w.r.t. y_vec (large init): tensor([6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09,
        6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09])
New y_vec after one gradient step: tensor([9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936,
        9.9936], grad_fn=<SubBackward0>)


In [424]:
# what if we make y_vec small?
y_vec_small = torch.tensor([p_min for i in range(K)], dtype=torch.float32, requires_grad=True)

loss_small = simulate_one_driver(y_vec_small, prices, base_demand_all, flex_demand_all)
loss_small.backward()
print("Gradients of cost w.r.t. y_vec (small init):", y_vec_small.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_small - lr * y_vec_small.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 1.0]
Cost of this solution: 14.0
Gradients of cost w.r.t. y_vec (small init): tensor([3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10,
        3.5394e-10, 3.5394e-10, 3.5394e-10, 3.5394e-10])
New y_vec after one gradient step: tensor([0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996, 0.9996,
        0.9996], grad_fn=<SubBackward0>)


In [425]:
# with penalty and ridge
pald_base_layer = make_pald_base_layer(K, gamma, ridge=True)
y_vec_large = torch.tensor([p_max for i in range(K)], dtype=torch.float32, requires_grad=True)
# Run and backprop to inspect gradients w.r.t y_vec
loss = simulate_one_driver_penalty(y_vec_large, prices, base_demand_all, flex_demand_all)
loss.backward()
print("Gradients of augmentedcost w.r.t. y_vec (large init):", y_vec_large.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_large - lr * y_vec_large.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [0.0, 0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Augmented cost of this solution: 10.75
Gradients of augmentedcost w.r.t. y_vec (large init): tensor([6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09,
        6.4438e-09, 6.4438e-09, 6.4438e-09, 6.4438e-09])
New y_vec after one gradient step: tensor([9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936, 9.9936,
        9.9936], grad_fn=<SubBackward0>)


In [426]:
# what if we make y_vec small?
y_vec_small = torch.tensor([p_min for i in range(K)], dtype=torch.float32, requires_grad=True)

loss_small = simulate_one_driver_penalty(y_vec_small, prices, base_demand_all, flex_demand_all)
loss_small.backward()
print("Gradients of cost w.r.t. y_vec (small init):", y_vec_small.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 1_000_000
new_y_vec = y_vec_small - lr * y_vec_small.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 1.0]
Augmented cost of this solution: 214.0
Gradients of cost w.r.t. y_vec (small init): tensor([6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08,
        6.4807e-08, 6.4807e-08, 6.4807e-08, 6.4807e-08])
New y_vec after one gradient step: tensor([0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352,
        0.9352], grad_fn=<SubBackward0>)


In [427]:
import cvxpy as cp
import numpy as np

def pwq_from_decreasing_affine_samples(x, taus, y):
    """
    Build F(x) = -∫_0^x g(t) dt where g is piecewise affine, decreasing,
    specified by its values y at breakpoints taus (taus increasing).
    Normalization: F(0)=0 and F'(0)=-g(0).
    """
    taus = np.array(taus, dtype=float)
    y = np.array(y, dtype=float)
    # segment slopes of g
    a = (y[1:] - y[:-1]) / (taus[1:] - taus[:-1])
    # curvatures s = -a
    s = -a
    # hinge-squared weights: w0 = s0, wj = s_j - s_{j-1}
    w = np.empty_like(s)
    w[0] = s[0]
    w[1:] = s[1:] - s[:-1]
    # coefficients
    c0 = 0.0
    c1 = -y[0]  # F'(0) = -g(0)

    expr = c0 + c1 * x
    for tau, wi in zip(taus[:-1], w):  # no need for the last tau=1
        if wi != 0:
            expr += 0.5 * wi * cp.square(cp.pos(x - tau))
    return expr

# Example usage:
x1 = cp.Variable()
x2 = cp.Variable()
taus = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
y    = [10, 9.5, 8.5, 7.0, 5.0, 3.0]

F = pwq_from_decreasing_affine_samples(x, taus, y)

# compute the integral of the piecewise affine function defined by (taus, y) over [0, 1]
prob = cp.Problem(cp.Minimize(F), [x >= 0, x <= 0.8])
prob.solve(solver=cp.SCS)
print("Integral of piecewise affine function over [0, 1]:", prob.value)


Integral of piecewise affine function over [0, 1]: -6.50006821629744


/Users/adam/miniconda3/envs/pald/lib/python3.10/site-packages/scs/__init__.py:113: UserWarning: Converting P to a CSC (compressed sparse column) matrix; may take a while.
  warn(


In [428]:
import cvxpy as cp
import numpy as np

# ---- Define g via samples at breakpoints ----
taus = np.array([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])  # breakpoints τ_0..τ_K
y     = np.array([10.0, 9.5, 8.5, 7.0, 5.0, 3.0]) # g(τ_j)=y_j (decreasing)

p_t = 6.0                                        # price at this time step
gamma = 1.0                                      # switching cost parameter for x
x_prev = 0.0

# segment slopes of g and curvatures of F
a = (y[1:] - y[:-1]) / (taus[1:] - taus[:-1])   # slopes of g (≤ 0)
s = -a                                          # curvatures of F (≥ 0)
w = np.empty_like(s)
w[0]  = s[0]
w[1:] = s[1:] - s[:-1]                           # hinge-squared weights (≥ 0)

c0 = 0.0
c1 = -y[0]                                       # F'(τ_0) = -g(τ_0)

def F_expr(z):
    """Return CVXPY expression for F(z) = -∫_{τ0}^z g."""
    expr = c0 + c1 * z
    for tau, wi in zip(taus[:-1], w):            # no need for the last τ_K
        if wi != 0:
            expr += 0.5 * wi * cp.square(cp.pos(z - tau))
    return expr

# ---------- Single time step ----------
w_const = 0.4                                    # given
x = cp.Variable()                                # decision
z = w_const + x
obj_terms = [p_t * x, gamma * cp.abs(x - x_prev) + gamma * cp.abs(x), F_expr(z) - F_expr(w_const)]        # == -∫_{w}^{w+x} g

constraints = [z >= taus[0], z <= taus[-1],      # keep inside domain
               x >= 0.0, x <= taus[-1]-w_const]
prob = cp.Problem(cp.Minimize(cp.sum(obj_terms)), constraints)
prob.solve()

print("Optimal x:", x.value)
print("Optimal objective:", prob.value)



Optimal x: 0.0666666666666668
Optimal objective: -0.016666666666667496


In [429]:
import cvxpy as cp
import numpy as np

# ---- Define g via samples at breakpoints ----
taus = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])  # breakpoints τ_0..τ_K
y     = np.array([10.0, 9.95, 9.9, 9.85, 9.8, 9.4, 9.0, 8.5, 8.0, 7.5, 7.0]) # g(τ_j)=y_j (decreasing)

p_t = 5.0                                        # price at this time step
gamma = 1.0                                      # switching cost parameter for x
x_prev = 0.0

# segment slopes of g and curvatures of F
a = (y[1:] - y[:-1]) / (taus[1:] - taus[:-1])   # slopes of g (≤ 0)
s = -a                                          # curvatures of F (≥ 0)
w = np.empty_like(s)
w[0]  = s[0]
w[1:] = s[1:] - s[:-1]                           # hinge-squared weights (≥ 0)
print(w)
# truncate w to be non-negative
w = np.maximum(w, 0)

c0 = 0.0
c1 = -y[0]                                       # F'(τ_0) = -g(τ_0)

def F_expr(z):
    """Return CVXPY expression for F(z) = -∫_{τ0}^z g."""
    expr = c0 + c1 * z
    for tau, wi in zip(taus[:-1], w):            # no need for the last τ_K
        if wi != 0:
            expr += 0.5 * wi * cp.square(cp.pos(z - tau))
    return expr

# ---------- Single time step ----------
w_const = 0.4                                    # given
x = cp.Variable()                                # decision
z = w_const + x
obj_terms = [p_t * x, gamma * cp.abs(x - x_prev) + gamma * cp.abs(x), F_expr(z) - F_expr(w_const)]        # == -∫_{w}^{w+x} g

constraints = [z >= taus[0], z <= taus[-1],      # keep inside domain
               x >= 0.0, x <= taus[-1]-w_const]
prob = cp.Problem(cp.Minimize(cp.sum(obj_terms)), constraints)
prob.solve()

print("Optimal x:", x.value)
print("Optimal objective:", prob.value)



[ 5.00000000e-01 -1.77635684e-14  1.78745907e-14 -1.80411242e-14
  3.50000000e+00  0.00000000e+00  1.00000000e+00 -5.32907052e-15
  5.32907052e-15  0.00000000e+00]
Optimal x: 0.5999975225082667
Optimal objective: -0.8799999999846451


In [430]:
# try making the layer
K=10
gamma = 1
layer = make_pald_base_layer(K, gamma)

In [438]:
import cvxpy as cp
import numpy as np
from cvxpylayers.torch import CvxpyLayer  # or .jax / .tf

# ---- compile-time constants ----
taus_full = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], dtype=float)
taus = taus_full[:-1]            # locations of hinge knots (exclude last)
tau0, tauK = float(taus_full[0]), float(taus_full[-1])
K = len(taus)

# ---- decision ----
x = cp.Variable()

# ---- parameters (layer inputs) ----
x_prev  = cp.Parameter(nonneg=True)                 # scalar
w_const = cp.Parameter(nonneg=True)                 # scalar
p_t     = cp.Parameter(nonneg=True)                 # scalar
gamma   = cp.Parameter(nonneg=True)      # scalar

# IMPORTANT: pass these precomputed from y (outside the graph)
w_hinge = cp.Parameter(K, nonneg=True)   # hinge weights >= 0
c1      = cp.Parameter()                 # slope term for F (e.g., -y[0])

# ---- auxiliaries (nonlinearities live here, no parameters inside atoms) ----
z = w_const + x
q = cp.Variable(K)                       # >= (z - tau_j)_+
r = cp.Variable(K)                       # >= 0.5 * q_j^2
u1 = cp.Variable()                       # >= |x - x_prev|
u2 = cp.Variable()                       # >= |x|

constraints = [
    z >= tau0, z <= tauK,
    x >= 0.0, x <= (tauK - w_const),

    q >= z - taus,
    q >= 0,

    cp.square(q) <= 2 * r,   # convex ≤ affine (DPP-safe)
    r >= 0,

    u1 >=  x - x_prev,  u1 >= -(x - x_prev),
    u2 >=  x,           u2 >= -x,
]

# IMPORTANT: no param*param products in the objective
# Use c1 * x (not c1 * z); the dropped c1*w_const is a parameter-only constant → irrelevant to argmin
Fz = c1 * x + w_hinge @ r

objective = cp.Minimize(p_t * x + gamma * (u1 + u2) + Fz)
prob = cp.Problem(objective, constraints)

layer = CvxpyLayer(
    prob,
    parameters=[x_prev, w_const, p_t, gamma, w_hinge, c1],
    variables=[x],
)



In [447]:
import torch
import numpy as np

def hinge_from_y_torch(taus_full_t: torch.Tensor, y: torch.Tensor):
    """
    taus_full_t: torch tensor of shape (K+1,), constant breakpoints [τ0,...,τK]
    y:           torch tensor of shape (K+1,) or (B, K+1), values g(τ_j)=y_j

    Returns:
      w_hinge: (K,) or (B, K), nonneg hinge weights
      c1:      ()  or (B,),   slope term for F; c1 = -y[..., 0]
    """
    # Ensure taus on same device/dtype as y
    taus_full_t = taus_full_t.to(device=y.device, dtype=y.dtype)

    # Differences along the last dimension
    dt = taus_full_t[1:] - taus_full_t[:-1]                     # (K,)
    dy = y[..., 1:] - y[..., :-1]                               # (..., K)

    # Slopes of g on segments, then curvatures of F
    a = dy / dt                                                 # (..., K)
    s = -a                                                      # (..., K)

    # Hinge weights are curvature jumps
    w0 = s[..., :1]                                             # (..., 1)
    wj = s[..., 1:] - s[..., :-1]                               # (..., K-1)
    w = torch.cat([w0, wj], dim=-1)                             # (..., K)

    # Nonnegativity projection; subgradients pass where w>0
    # w = torch.clamp(w, min=0.0)

    # c1 = -g(τ0) = -y[..., 0]
    c1 = -y[..., 0]

    return w, c1


# Assume you already constructed `layer = CvxpyLayer(...)` as before.

# Constant breakpoints as a torch tensor
taus_full_t = torch.linspace(0.0, 1.0, 11, dtype=torch.float32)

# Learnable y
y_torch = torch.nn.Parameter(torch.tensor(
    [10.0, 9.95, 9.9, 9.85, 9.8, 9.4, 9.0, 8.5, 8.0, 7.5, 7.0],
    dtype=torch.float32
))

# Other inputs (scalars)
x_prev_t  = torch.tensor(0.0)
w_const_t = torch.tensor(0.0)
p_t_t     = torch.tensor(7.0)
gamma_t   = torch.tensor(1.0)

# === forward ===
w_hinge_t, c1_t = hinge_from_y_torch(taus_full_t, y_torch)

# Call the layer
(x_star,) = layer(x_prev_t, w_const_t, p_t_t, gamma_t, w_hinge_t, c1_t)

# Example loss using x_star; gradients will flow back to y_torch
loss = 0.5 * x_star.pow(2)
loss.backward()
# y_torch.grad now populated


In [ ]:
# Call the layer
x_prev_t = torch.tensor(0.0, requires_grad=True)
w_const_t = torch.tensor(0.0)
p_t_t     = torch.tensor(20.9250)
gamma_t   = torch.tensor(10.0)
# Learnable y
y_torch = torch.nn.Parameter(torch.tensor(
    [100,99,97,94,90,85,79,72,64,55,45],
    dtype=torch.float32
))
# print differences between consecutive elements of y_torch
print(torch.diff(y_torch))

w_hinge_t, c1_t = hinge_from_y_torch(taus_full_t, y_torch)
print(w_hinge_t) # ensure that w_hinge is non-negative somehow
print(c1_t)
(x_star,) = layer(x_prev_t, w_const_t, p_t_t, gamma_t, w_hinge_t, c1_t)

tensor([ -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10.],
       grad_fn=<SubBackward0>)
tensor([10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,
        10.0000,  9.9999], grad_fn=<CatBackward0>)
tensor(-100., grad_fn=<NegBackward0>)


In [460]:
# print x_star
print("x_star: ", x_star)

x_star:  tensor(4.7275e-06, grad_fn=<_CvxpyLayerFnFnBackward>)


## retrying the gradient simulation with the new hinge implementation of the cvxpy layer

In [434]:
def make_pald_base_layer(K, gamma):
    # ---- compile-time constants ----
    taus_full = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], dtype=float)
    taus = taus_full[:-1]            # locations of hinge knots (exclude last)
    tau0, tauK = float(taus_full[0]), float(taus_full[-1])

    # ---- decision ----
    x = cp.Variable()

    # ---- parameters (layer inputs) ----
    x_prev  = cp.Parameter(1,nonneg=True)                 # scalar
    w_const = cp.Parameter(1,nonneg=True)                 # scalar
    p_t     = cp.Parameter(1,nonneg=True)                 # scalar
    gamma   = cp.Parameter(1,nonneg=True)                 # scalar
    # IMPORTANT: pass these precomputed from y (outside the graph)
    w_hinge = cp.Parameter(K, nonneg=True)   # hinge weights >= 0
    c1      = cp.Parameter()                 # slope term for F (e.g., -y[0])

    # ---- auxiliaries (nonlinearities live here, no parameters inside atoms) ----
    z = w_const + x
    q = cp.Variable(K)                       # >= (z - tau_j)_+
    r = cp.Variable(K)                       # >= 0.5 * q_j^2
    u1 = cp.Variable()                       # >= |x - x_prev|
    u2 = cp.Variable()                       # >= |x|

    constraints = [
        z >= tau0, z <= tauK,
        x >= 0.0, x <= (tauK - w_const),

        q >= z - taus,
        q >= 0,

        cp.square(q) <= 2 * r,   # convex ≤ affine (DPP-safe)
        r >= 0,

        u1 >=  x - x_prev,  u1 >= -(x - x_prev),
        u2 >=  x,           u2 >= -x,
    ]

    # IMPORTANT: no param*param products in the objective
    # Use c1 * x (not c1 * z); the dropped c1*w_const is a parameter-only constant → irrelevant to argmin
    Fz = c1 * x + w_hinge @ r

    objective = cp.Minimize(p_t * x + gamma * (u1 + u2) + Fz)
    prob = cp.Problem(objective, constraints)

    layer = CvxpyLayer(
        prob,
        parameters=[x_prev, w_const, p_t, gamma, w_hinge, c1],
        variables=[x],
    )

    return layer



In [435]:
K = 10           # number of segments in piecewise linear approximation for psi
gamma = 1.0     # switching cost parameter for x
delta = 0.5     # switching cost parameter for z (used in analytical threshold)
S = 1.0          # maximum inventory capacity
c_delivery = 0.2
eps_delivery = 0.05
epochs = 10
T = 10

# define a toy instance of osdm where prices decrease from 10 to 1 and then jump up to 10, 
# single unit of base demand at T=10, no flex demand
p_min = 1.0
p_max = 10.0
base_demand_all = [1.0 if t == T-1 else 0.0 for t in range(T)]
flex_demand_all = [0.0 for t in range(T)]

prices = np.linspace(p_max, p_min, T-1).tolist()
prices.append(p_max)

solver_options = {
    # SCS parameters tend to be robust for differentiable layers
    "eps": 1e-5,
    "max_iters": 2000,
    "verbose": False,
}


def _safe_layer_call(layer, args, size=1.0):
    """
    Call a CvxpyLayer and catch SCS/diffcp failures. Returns x_total tensor.
    """
    (x_total,) = layer(*args, solver_args=solver_options)
    return x_total

In [436]:
# define the base demand cvxpylayer
pald_base_layer = make_pald_base_layer(K, gamma)

# y_vec is the values of the piecewise affine threshold function -- let's see what happens if we set it HIGH
# IMPORTANT: To get valid gradients w.r.t. y_vec, we must keep decisions as torch Tensors and avoid .item()/.round()
y_vec_large = torch.tensor([p_max - (i*1e-5) for i in range(K+1)], dtype=torch.float32, requires_grad=True)


def simulate_one_driver(y_vec, prices, base_demand_all, flex_demand_all, ridge=False):
    """
    Simulate one base driver with given y_vec and return a differentiable loss.
    Decisions remain as torch.Tensors so autograd can backprop to y_vec through the CvxpyLayer.
    """
    # torch state (kept differentiable across time)
    x_prev = torch.tensor(0.0, dtype=torch.float32).reshape(-1)  # previous decision (scalar)
    w_prev = torch.tensor(0.0, dtype=torch.float32).reshape(-1)  # cumulative fraction delivered
    gamma_t = torch.tensor(gamma, dtype=torch.float32).reshape(-1)

    torch_decisions = []

    for t in range(T):
        p_t = prices[t]
        # Prepare parameters for the layer
        p_t_t     = torch.tensor([p_t], dtype=torch.float32)

        # Assume you already constructed `layer = CvxpyLayer(...)` as before.

        # Constant breakpoints as a torch tensor
        taus_full_t = torch.linspace(0.0, 1.0, 11, dtype=torch.float32)

        # === forward ===
        w_hinge_t, c1_t = hinge_from_y_torch(taus_full_t, y_vec)

        # Solve the per-step convex problem via CVXPYLayer (returns x_total, x_parts)
        x_total_t = _safe_layer_call(
            pald_base_layer, (x_prev, w_prev, p_t_t, gamma_t, w_hinge_t, c1_t)
        )

        # On the final step, force completion if any remaining fraction < 1.0
        if t == T - 1:
            # Add just enough slack to finish any remainder (keeps graph intact)
            remainder = torch.clamp(1.0 - (w_prev + x_total_t), min=0.0)
            x_total_t = x_total_t + remainder

        torch_decisions.append(x_total_t.reshape(-1))

        # Update state for the next step (kept differentiable)
        w_prev = (w_prev + x_total_t).reshape(-1)
        x_prev = (x_total_t).reshape(-1)

    # Stack decisions and compute differentiable objective
    x_seq = torch.stack(torch_decisions)  # shape [T]
    z_seq = torch.tensor(base_demand_all, dtype=torch.float32)  # no flex here; constant

    delta_t = torch.tensor(delta, dtype=torch.float32)
    c_delivery_t = torch.tensor(c_delivery, dtype=torch.float32)
    eps_delivery_t = torch.tensor(eps_delivery, dtype=torch.float32)

    loss = torch_objective(
        torch.tensor(prices, dtype=torch.float32),
        x_seq,
        z_seq,
        gamma_t,
        delta_t,
        c_delivery_t,
        eps_delivery_t,
    )

    # For visibility only (no graph break for the loss)
    with torch.no_grad():
        pretty = [round(float(v), 4) for v in x_seq]
        print("Decisions:", pretty)
        print("Cost of this solution:", float(loss))

    return loss



# Run and backprop to inspect gradients w.r.t y_vec
loss = simulate_one_driver(y_vec_large, prices, base_demand_all, flex_demand_all)
loss.backward()
print("Gradients of cost w.r.t. y_vec (large init):", y_vec_large.grad)
# take one step in the direction of the gradient to see which direction it's going to take us
lr = 0.001
new_y_vec = y_vec_large - lr * y_vec_large.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0]
Cost of this solution: 62.50201416015625
Gradients of cost w.r.t. y_vec (large init): tensor([ 133.5214,  -42.3683, -215.2831,  248.2735, -274.9716,  301.6568,
        -301.5762,  301.4978, -300.7552,  226.4803,  -76.4755])
New y_vec after one gradient step: tensor([ 9.8665, 10.0424, 10.2153,  9.7517, 10.2749,  9.6983, 10.3015,  9.6984,
        10.3007,  9.7734, 10.0764], grad_fn=<SubBackward0>)


In [437]:
# what if we make y_vec small?
y_vec_small = torch.tensor([p_min + ((K-i)*1e-5) for i in range(K+1)], dtype=torch.float32, requires_grad=True)


loss_small = simulate_one_driver(y_vec_small, prices, base_demand_all, flex_demand_all)
loss_small.backward()
print("Gradients of cost w.r.t. y_vec (small init):", y_vec_small.grad)
# take one big step in the direction of the gradient to see which direction it's going to take us
lr = 0.001
new_y_vec = y_vec_small - lr * y_vec_small.grad
print("New y_vec after one gradient step:", new_y_vec)

Decisions: [-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Cost of this solution: 63.4999885559082
Gradients of cost w.r.t. y_vec (small init): tensor([-5.9344e-01,  5.9169e-01,  1.2169e-03,  1.3809e-03, -1.1238e+00,
         3.3742e+00, -3.3711e+00,  1.1240e+00,  1.5344e-04, -2.0667e-01,
         2.0237e-01])
New y_vec after one gradient step: tensor([1.0007, 0.9995, 1.0001, 1.0001, 1.0012, 0.9967, 1.0034, 0.9989, 1.0000,
        1.0002, 0.9998], grad_fn=<SubBackward0>)
